In [ ]:
from Transformers import  BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text="Hello,how are you?"
#tokenize and encode the text 

encoding=tokenizer.encode(text)
print("Token IDs:",encoding)

#covert token ids back to text
decoded_text=tokenizer.decode(encoding)
print("Decoded Text:",decoded_text)